# First Run of Results Tracker

In [2]:
import pandas as pd
import datetime as dt
import re

current_year = dt.date.today().year
today = str(dt.date.today()).replace('-', '')
today_date = dt.date.today()
yesterday = today_date - dt.timedelta(days=1)
yesterday_string = str(yesterday)
yesterday_string = yesterday_string.replace('-', '')

In [3]:
team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians' : 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

In [13]:
def calculate_yesterdays_bet_results_external(yesterday_string, yesterdays_capital_538):    
    def calculate_payoff_external(row):
        if row.Bet_538<=0:
            payoff = 0
        else:
            if row.Home_KC_538>0:
                if row.Home_Odds>0:
                    payoff = (row.Home_Odds/100)*row.Bet_538
                if row.Home_Odds<0:
                    payoff = row.Bet_538/((abs(row.Home_Odds)/100))
            if row.Away_KC_538>0:
                if row.Away_Odds>0:
                    payoff = (row.Away_Odds/100)*row.Bet_538
                if row.Away_Odds<0:
                    payoff = row.Bet_538/((abs(row.Away_Odds)/100))
        return payoff

    # Getting yesterdays results from CBS
    link = 'https://www.cbssports.com/mlb/scoreboard/' + yesterday_string + '/'
    tables = pd.read_html(link)
    results_table = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Winner'])
    team_regex = r'\D*'
    for table in tables:
        if list(table.columns) == ['Unnamed: 0', 'R', 'H', 'E']:

            # Getting team names
            team_away_string = table.iloc[0,0]
            team_away_list = re.findall(team_regex, team_away_string)
            team_away = team_away_list[0]
            
            team_home_string = table.iloc[1,0]
            team_home_list = re.findall(team_regex, team_home_string)
            team_home = team_home_list[0]
            
            # Getting score and determining winner
            runs_away = table.iloc[0,1]
            runs_home = table.iloc[1,1]
            if runs_away>runs_home:
                winner = team_away
            else:
                winner = team_home
            
            # Appending to results table
            series = pd.Series([team_home, team_away, winner], index = results_table.columns)
            results_table = results_table.append(series, ignore_index = True)        
        else:
            continue
    for column in list(results_table.columns):
        results_table[column] = results_table[column].apply(lambda x: team_map[x])



    # Reading in yesterdays bets and creating tracker columns
    yesterdays_bets = pd.read_csv('past_bets/external/bets_' + yesterday_string + '.csv', index_col = 0)
    yesterdays_bets = yesterdays_bets[(yesterdays_bets.Home_Team != '538') & (yesterdays_bets.Away_Team != '538')]
    yesterdays_bets['Home_Team'] = yesterdays_bets['Home_Team'].apply(lambda x: team_map[x])
    yesterdays_bets['Away_Team'] = yesterdays_bets['Away_Team'].apply(lambda x: team_map[x])
    yesterdays_bets = yesterdays_bets[yesterdays_bets.Bet_538>0]
    yesterdays_bets.reset_index(drop = True, inplace = True)
    yesterdays_bets['Won_538'] = 0
    yesterdays_bets['Tracker_538'] = 0
    for index, row in yesterdays_bets.iterrows():
        home_team = row.Home_Team
        away_team = row.Away_Team
        payoff_538 = calculate_payoff_external(row)
        # Signifying win, loss, or other
        if row.Bet_538>0:
            # Normal bets
            if (row.Home_KC_538>0) & (home_team in results_table['Winner'].values):
                yesterdays_bets.loc[index, 'Won_538'] = 1
            elif (row.Away_KC_538>0) & (away_team in results_table['Winner'].values):
                yesterdays_bets.loc[index, 'Won_538'] = 1
            else:
                yesterdays_bets.loc[index, 'Won_538'] = 0
            # Postponed games
            if (home_team not in results_table['Winner'].values) & (away_team not in results_table['Winner'].values):
                yesterdays_bets.loc[index, 'Won_538'] = -1
            # Double headers
            if (home_team in results_table['Winner'].values) & (away_team in results_table['Winner'].values):
                yesterdays_bets.loc[index, 'Won_538'] = -1
            elif (list(results_table['Winner'].values).count(home_team) > 1) | (list(results_table['Winner'].values).count(away_team) > 1):
                yesterdays_bets.loc[index, 'Won_538'] = -1
        # Games that weren't bet on
        else:
            yesterdays_bets.loc[index, 'Won_538'] = -1
        # Updating tracker
        if yesterdays_bets.loc[index, 'Won_538'] == 1:
            if index == 0:
                yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538 + payoff_538
            else:
                yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538'] + payoff_538
        elif yesterdays_bets.loc[index, 'Won_538'] == -1:
            if index == 0:
                yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538
            else:
                yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538']
        else:
            if index == 0:
                yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538 - row.Bet_538
            else:
                yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538'] - row.Bet_538
            
    return yesterdays_bets

In [14]:
yesterdays_bets = calculate_yesterdays_bet_results_external(yesterday_string, 100000)

/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_14943/1890174374.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append(series, ignore_index = True)
/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_14943/1890174374.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append(series, ignore_index = True)
/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_14943/1890174374.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append(series, ignore_index = True)
/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_14943/1890174374.py:45: FutureWarning: The frame.append method is deprecated and will be 

In [16]:
yesterdays_bets.to_csv('results_tracker/results_tracker_external.csv')